In [23]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import os
os.environ["OMP_NUM_THREADS"] = "1"
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)

In [24]:
# Load processed sales data from ETL step
df = pd.read_csv("data_warehouse/processed_sales_data.csv")

In [25]:
# ----------------------------
# Step 1: Aggregate customer features
# ----------------------------
customer_df = (
    df.groupby('customer_id')
    .agg(
        total_purchase=('total_revenue', 'sum'),
        purchase_frequency=('product_id', 'count'),
        avg_transaction_value=('total_revenue', 'mean')
    )
    .reset_index()
)

In [26]:
# ----------------------------
# Step 2: Preprocessing
# ----------------------------
features = customer_df[['total_purchase', 'purchase_frequency', 'avg_transaction_value']]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)

In [27]:
# ----------------------------
# Step 3: K-Means clustering
# ----------------------------
kmeans = KMeans(n_clusters=2, random_state=42)
customer_df['cluster'] = kmeans.fit_predict(X_scaled)

# Identify which cluster is VIP (higher total purchase)
vip_cluster = customer_df.groupby('cluster')['total_purchase'].mean().idxmax()
customer_df['VIP_status'] = customer_df['cluster'].apply(
    lambda x: 'VIP' if x == vip_cluster else 'Non-VIP'
)

In [28]:
# ----------------------------
# Step 4: Reverse ETL
# ----------------------------
df = df.merge(customer_df[['customer_id', 'VIP_status']], on='customer_id', how='left')

In [29]:
# Save enriched dataset
df.to_csv("data_warehouse/processed_sales_with_vip.csv", index=False)
print("Updated dataset with VIP status saved to data_warehouse/processed_sales_with_vip.csv")

# Optional: quick check
print(df[['customer_id', 'VIP_status']].drop_duplicates().head())

Updated dataset with VIP status saved to data_warehouse/processed_sales_with_vip.csv
  customer_id VIP_status
0       C_001    Non-VIP
1       C_002    Non-VIP
2       C_003    Non-VIP
3       C_004    Non-VIP
4       C_005    Non-VIP
